<a href="https://colab.research.google.com/github/aekanun2020/2025-AdvancedRAG/blob/main/SENT_Hands_on_3_Search_by_Retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ติดตั้ง LlamaIndex และ dependencies
!pip install llama-index -q
!pip install llama-index-embeddings-huggingface -q
!pip install llama-index-vector-stores-opensearch -q
!pip install requests -q
!pip install nest_asyncio -q

# Import modules
import os
import torch
import urllib.request
import pickle
import requests
import nest_asyncio
import json
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.opensearch import OpensearchVectorStore, OpensearchVectorClient
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from typing import List
import pandas as pd
from IPython.display import display, HTML
import time

# Apply nest_asyncio to avoid runtime errors
nest_asyncio.apply()

# กำหนดค่าสำหรับ OpenSearch
OPENSEARCH_ENDPOINT = "http://34.41.37.53:9200"
OPENSEARCH_INDEX = "aekanun_doc_index"
TEXT_FIELD = "content"
EMBEDDING_FIELD = "embedding"

# Check if CUDA is available for GPU acceleration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"ใช้อุปกรณ์: {device}")


# ฟังก์ชันค้นหาข้อมูลโดยไม่ตัดทอนผลลัพธ์
def search_without_truncation(query: str, top_k: int = 5):
    """
    ค้นหาข้อมูลจาก index โดยไม่ตัดทอนผลลัพธ์
    :param query: คำค้นหา
    :param top_k: จำนวนผลลัพธ์สูงสุดที่ต้องการ
    :return: ผลลัพธ์การค้นหา
    """
    # สร้าง retriever แบบไม่ตัดทอนผลลัพธ์
    retriever = index.as_retriever(
        similarity_top_k=top_k,
        vector_store_query_mode=VectorStoreQueryMode.HYBRID
    )

    # ค้นหาข้อมูล
    results = retriever.retrieve(query)

    # เตรียมข้อมูลสำหรับแสดงผล
    formatted_results = []

    for i, result in enumerate(results):
        # Get full content of the node without truncation
        try:
            full_text = result.node.get_content()
        except AttributeError:
            full_text = getattr(result, "text", str(result))

        # เตรียมข้อมูล metadata
        metadata = result.metadata if hasattr(result, "metadata") else {}

        # เพิ่มข้อมูลในรายการผลลัพธ์
        formatted_results.append({
            "rank": i + 1,
            "score": result.score if hasattr(result, "score") else None,
            "text": full_text,
            "metadata": metadata
        })

    return formatted_results

# ฟังก์ชันแสดงผลลัพธ์แบบสวยงาม
def display_search_results(results: List[dict]):
    """
    แสดงผลลัพธ์การค้นหาในรูปแบบที่อ่านง่าย
    :param results: ผลลัพธ์การค้นหาที่ได้จาก search_without_truncation
    """
    for i, result in enumerate(results):
        print(f"\n{'='*80}")
        print(f"ผลลัพธ์ที่ {result['rank']} (คะแนน: {result['score']:.4f})" if result['score'] else f"ผลลัพธ์ที่ {result['rank']}")
        print(f"{'-'*80}")

        # แสดง metadata ทั้งหมด
        print("Metadata:")
        for key, value in result["metadata"].items():
            print(f"  - {key}: {value}")

        print(f"{'-'*80}")
        print("เนื้อหา:")
        print(f"{result['text']}")

    print(f"\n{'='*80}")
    print(f"พบผลลัพธ์ทั้งหมด {len(results)} รายการ")

# คำถามที่กำหนดไว้ล่วงหน้า
predefined_questions = [
        "โรคหัดและโรคหัดเยอรมันแตกต่างกันอย่างไร?",  # คำถามเปรียบเทียบ
        "อธิบายสาเหตุของโรคหัดเยอรมันและการป้องกัน",  # คำถามหลายประเด็น
        "ทำไมโรคหัดเยอรมันจึงมีอันตรายกับหญิงตั้งครรภ์?",  # คำถามวิเคราะห์เชิงลึก
        "ถ้าคนที่ฉีดวัคซีนป้องกันโรคหัดเยอรมันแล้ว จะมีโอกาสติดเชื้อหรือไม่?",  # คำถามสมมติเหตุการณ์
        "โรคหัดเยอรมันมีผลกระทบอย่างไรต่อระบบสาธารณสุขและเศรษฐกิจของประเทศ?",  # คำถามข้ามสาขา
        "การรักษาโรคหัดเยอรมันที่ดีที่สุดคืออะไร?"  # คำถามกำกวม
    ]


# ดำเนินการหลัก
print("กำลังเริ่มต้นระบบค้นหาข้อมูลสุขภาพ...")


## ตั้งค่า embedding model (ไม่ได้มีไว้เพื่อ embedding คำถามหรือเอกสาร
## แต่มีไว้ทดลอง embedding "test" และ get ขนาดของ dim)
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3", device=device)
print(f"ตั้งค่าโมเดล embedding BAAI/bge-m3 สำเร็จ")

# ตรวจสอบขนาดของ embedding
embeddings = embed_model.get_text_embedding("test")
dim = len(embeddings)
print(f"ขนาด embedding: {dim}")

# ตั้งค่า OpensearchVectorClient
client = OpensearchVectorClient(
    endpoint=OPENSEARCH_ENDPOINT,
    index=OPENSEARCH_INDEX,
    dim=dim,
    embedding_field=EMBEDDING_FIELD,
    text_field=TEXT_FIELD,
    search_pipeline="hybrid-search-pipeline",
)
print(f"ตั้งค่า OpensearchVectorClient สำเร็จ สำหรับ index '{OPENSEARCH_INDEX}'")

# สร้าง vector store
vector_store = OpensearchVectorStore(client)

# สร้าง index จาก vector store ที่มีอยู่แล้ว
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model
)

# บันทึก index ด้วย pickle
##index_filename = f"{OPENSEARCH_INDEX}.pkl"
##with open(index_filename, 'wb') as f:
##    pickle.dump(index, f)
##print(f"บันทึก index ลงในไฟล์ {index_filename} สำเร็จ")

# ทดสอบการค้นหาด้วยคำถามที่กำหนดไว้ล่วงหน้า
print("\nเริ่มการทดสอบค้นหาด้วยคำถามที่กำหนดไว้ล่วงหน้า:")
print(f"{'-'*80}")

for i, question in enumerate(predefined_questions):
    print(f"\nคำถามที่ {i+1}: {question}")
    results = search_without_truncation(question)
    display_search_results(results)

    # ถามผู้ใช้ว่าต้องการดูคำถามถัดไปหรือไม่
    if i < len(predefined_questions) - 1:
        user_input = input("\nกด Enter เพื่อดูผลลัพธ์ของคำถามถัดไป หรือพิมพ์ 'q' เพื่อออก: ")
        if user_input.lower() == 'q':
            break

print("\nเสร็จสิ้นการค้นหา!")

# อนุญาตให้ผู้ใช้ป้อนคำถามเพิ่มเติม
print("\nคุณสามารถป้อนคำถามเพิ่มเติมได้:")
while True:
    user_question = input("\nป้อนคำถามของคุณ (หรือพิมพ์ 'exit' เพื่อออก): ")
    if user_question.lower() == 'exit':
        break

    results = search_without_truncation(user_question)
    display_search_results(results)

print("\nขอบคุณที่ใช้ระบบค้นหาข้อมูลสุขภาพ!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

ตั้งค่าโมเดล embedding BAAI/bge-m3 สำเร็จ
ขนาด embedding: 1024
ตั้งค่า OpensearchVectorClient สำเร็จ สำหรับ index 'aekanun_doc_index'

เริ่มการทดสอบค้นหาด้วยคำถามที่กำหนดไว้ล่วงหน้า:
--------------------------------------------------------------------------------

คำถามที่ 1: โรคหัดและโรคหัดเยอรมันแตกต่างกันอย่างไร?

ผลลัพธ์ที่ 1 (คะแนน: 1.0000)
--------------------------------------------------------------------------------
Metadata:
  - file_path: /content/corpus_input/1.md
  - file_name: 1.md
  - file_type: text/markdown
  - file_size: 65320
  - creation_date: 2025-03-18
  - last_modified_date: 2025-03-18
  - header_path: /หัดเยอรมัน อาการ สาเหตุ และการรักษาโรคหัดเยอรมัน 10 วิธี !!/
--------------------------------------------------------------------------------
เนื้อหา:
## โรคหัดเยอรมัน

หัดเยอรมัน, เหือด หรือ หัดสามวัน (German measles/เจอร์มันมีเซิลส์, Rubella/รูเบลลา หรือ Three-day measles/ทรีเดย์มีเซิลส์) เป็นโรคไข้ออกผื่นที่เกิดจากการติดเชื้อไวรัสหัดเยอรมัน ผู้ป่วยจะมีอาการไข้แ